# Process sequencing-based neutralization assay plate
This notebook is designed to be run using `snakemake`.

## Setup
Import Python modules:

In [1]:
import altair as alt

import neutcurve

import pandas as pd

_ = alt.data_transformers.disable_max_rows()

Get the variables passed by `snakemake`:

In [2]:

######## snakemake preamble start (automatically inserted, do not edit) ########
import sys; sys.path.extend(['/fh/fast/bloom_j/software/miniconda3/envs/seqneut-pipeline/lib/python3.11/site-packages', '/home/jbloom/.cache/snakemake/snakemake/source-cache/runtime-cache/tmpcade3ue1/file/fh/fast/bloom_j/computational_notebooks/jbloom/2023/flu_seqneut_DRIVE_2021-22_repeat_vax/seqneut-pipeline/notebooks', '/fh/fast/bloom_j/computational_notebooks/jbloom/2023/flu_seqneut_DRIVE_2021-22_repeat_vax/seqneut-pipeline/notebooks']); import pickle; snakemake = pickle.loads(b'\x80\x04\x95\xf8\x9d\x00\x00\x00\x00\x00\x00\x8c\x10snakemake.script\x94\x8c\tSnakemake\x94\x93\x94)\x81\x94}\x94(\x8c\x05input\x94\x8c\x0csnakemake.io\x94\x8c\nInputFiles\x94\x93\x94)\x81\x94(\x8c+results/barcode_counts/plate2_M099d0_20.csv\x94\x8c+results/barcode_counts/plate2_M099d0_60.csv\x94\x8c,results/barcode_counts/plate2_M099d0_180.csv\x94\x8c,results/barcode_counts/plate2_M099d0_540.csv\x94\x8c-results/barcode_counts/plate2_M099d0_1620.csv\x94\x8c-results/barcode_counts/plate2_M099d0_4860.csv\x94\x8c.results/barcode_counts/plate2_M099d0_14580.csv\x94\x8c.results/barcode_counts/plate2_M099d0_43740.csv\x94\x8c/results/barcode_counts/plate2_M099d0_131220.csv\x94\x8c/results/barcode_counts/plate2_M099d0_393660.csv\x94\x8c-results/barcode_counts/plate2_M099d182_20.csv\x94\x8c-results/barcode_counts/plate2_M099d182_60.csv\x94\x8c.results/barcode_counts/plate2_M099d182_180.csv\x94\x8c.results/barcode_counts/plate2_M099d182_540.csv\x94\x8c/results/barcode_counts/plate2_M099d182_1620.csv\x94\x8c/results/barcode_counts/plate2_M099d182_4860.csv\x94\x8c0results/barcode_counts/plate2_M099d182_14580.csv\x94\x8c0results/barcode_counts/plate2_M099d182_43740.csv\x94\x8c1results/barcode_counts/plate2_M099d182_131220.csv\x94\x8c1results/barcode_counts/plate2_M099d182_393660.csv\x94\x8c,results/barcode_counts/plate2_M099d30_20.csv\x94\x8c,results/barcode_counts/plate2_M099d30_60.csv\x94\x8c-results/barcode_counts/plate2_M099d30_180.csv\x94\x8c-results/barcode_counts/plate2_M099d30_540.csv\x94\x8c.results/barcode_counts/plate2_M099d30_1620.csv\x94\x8c.results/barcode_counts/plate2_M099d30_4860.csv\x94\x8c/results/barcode_counts/plate2_M099d30_14580.csv\x94\x8c/results/barcode_counts/plate2_M099d30_43740.csv\x94\x8c0results/barcode_counts/plate2_M099d30_131220.csv\x94\x8c0results/barcode_counts/plate2_M099d30_393660.csv\x94\x8c+results/barcode_counts/plate2_Y008d0_20.csv\x94\x8c+results/barcode_counts/plate2_Y008d0_60.csv\x94\x8c,results/barcode_counts/plate2_Y008d0_180.csv\x94\x8c,results/barcode_counts/plate2_Y008d0_540.csv\x94\x8c-results/barcode_counts/plate2_Y008d0_1620.csv\x94\x8c-results/barcode_counts/plate2_Y008d0_4860.csv\x94\x8c.results/barcode_counts/plate2_Y008d0_14580.csv\x94\x8c.results/barcode_counts/plate2_Y008d0_43740.csv\x94\x8c/results/barcode_counts/plate2_Y008d0_131220.csv\x94\x8c/results/barcode_counts/plate2_Y008d0_393660.csv\x94\x8c,results/barcode_counts/plate2_Y008d30_20.csv\x94\x8c,results/barcode_counts/plate2_Y008d30_60.csv\x94\x8c-results/barcode_counts/plate2_Y008d30_180.csv\x94\x8c-results/barcode_counts/plate2_Y008d30_540.csv\x94\x8c.results/barcode_counts/plate2_Y008d30_1620.csv\x94\x8c.results/barcode_counts/plate2_Y008d30_4860.csv\x94\x8c/results/barcode_counts/plate2_Y008d30_14580.csv\x94\x8c/results/barcode_counts/plate2_Y008d30_43740.csv\x94\x8c0results/barcode_counts/plate2_Y008d30_131220.csv\x94\x8c0results/barcode_counts/plate2_Y008d30_393660.csv\x94\x8c+results/barcode_counts/plate2_Y154d0_20.csv\x94\x8c+results/barcode_counts/plate2_Y154d0_60.csv\x94\x8c,results/barcode_counts/plate2_Y154d0_180.csv\x94\x8c,results/barcode_counts/plate2_Y154d0_540.csv\x94\x8c-results/barcode_counts/plate2_Y154d0_1620.csv\x94\x8c-results/barcode_counts/plate2_Y154d0_4860.csv\x94\x8c.results/barcode_counts/plate2_Y154d0_14580.csv\x94\x8c.results/barcode_counts/plate2_Y154d0_43740.csv\x94\x8c/results/barcode_counts/plate2_Y154d0_131220.csv\x94\x8c/results/barcode_counts/plate2_Y154d0_393660.csv\x94\x8c-results/barcode_counts/plate2_Y154d182_20.csv\x94\x8c-results/barcode_counts/plate2_Y154d182_60.csv\x94\x8c.results/barcode_counts/plate2_Y154d182_180.csv\x94\x8c.results/barcode_counts/plate2_Y154d182_540.csv\x94\x8c/results/barcode_counts/plate2_Y154d182_1620.csv\x94\x8c/results/barcode_counts/plate2_Y154d182_4860.csv\x94\x8c0results/barcode_counts/plate2_Y154d182_14580.csv\x94\x8c0results/barcode_counts/plate2_Y154d182_43740.csv\x94\x8c1results/barcode_counts/plate2_Y154d182_131220.csv\x94\x8c1results/barcode_counts/plate2_Y154d182_393660.csv\x94\x8c,results/barcode_counts/plate2_Y154d30_20.csv\x94\x8c,results/barcode_counts/plate2_Y154d30_60.csv\x94\x8c-results/barcode_counts/plate2_Y154d30_180.csv\x94\x8c-results/barcode_counts/plate2_Y154d30_540.csv\x94\x8c.results/barcode_counts/plate2_Y154d30_1620.csv\x94\x8c.results/barcode_counts/plate2_Y154d30_4860.csv\x94\x8c/results/barcode_counts/plate2_Y154d30_14580.csv\x94\x8c/results/barcode_counts/plate2_Y154d30_43740.csv\x94\x8c0results/barcode_counts/plate2_Y154d30_131220.csv\x94\x8c0results/barcode_counts/plate2_Y154d30_393660.csv\x94\x8c(results/barcode_counts/plate2_none-1.csv\x94\x8c(results/barcode_counts/plate2_none-2.csv\x94\x8c(results/barcode_counts/plate2_none-3.csv\x94\x8c(results/barcode_counts/plate2_none-4.csv\x94\x8c(results/barcode_counts/plate2_none-5.csv\x94\x8c(results/barcode_counts/plate2_none-6.csv\x94\x8c(results/barcode_counts/plate2_none-7.csv\x94\x8c(results/barcode_counts/plate2_none-8.csv\x94\x8c(results/barcode_counts/plate2_none-9.csv\x94\x8c)results/barcode_counts/plate2_none-10.csv\x94\x8c)results/barcode_counts/plate2_none-11.csv\x94\x8c)results/barcode_counts/plate2_none-12.csv\x94\x8c)results/barcode_counts/plate2_none-13.csv\x94\x8c)results/barcode_counts/plate2_none-14.csv\x94\x8c)results/barcode_counts/plate2_none-15.csv\x94\x8c)results/barcode_counts/plate2_none-16.csv\x94\x8c*results/barcode_fates/plate2_M099d0_20.csv\x94\x8c*results/barcode_fates/plate2_M099d0_60.csv\x94\x8c+results/barcode_fates/plate2_M099d0_180.csv\x94\x8c+results/barcode_fates/plate2_M099d0_540.csv\x94\x8c,results/barcode_fates/plate2_M099d0_1620.csv\x94\x8c,results/barcode_fates/plate2_M099d0_4860.csv\x94\x8c-results/barcode_fates/plate2_M099d0_14580.csv\x94\x8c-results/barcode_fates/plate2_M099d0_43740.csv\x94\x8c.results/barcode_fates/plate2_M099d0_131220.csv\x94\x8c.results/barcode_fates/plate2_M099d0_393660.csv\x94\x8c,results/barcode_fates/plate2_M099d182_20.csv\x94\x8c,results/barcode_fates/plate2_M099d182_60.csv\x94\x8c-results/barcode_fates/plate2_M099d182_180.csv\x94\x8c-results/barcode_fates/plate2_M099d182_540.csv\x94\x8c.results/barcode_fates/plate2_M099d182_1620.csv\x94\x8c.results/barcode_fates/plate2_M099d182_4860.csv\x94\x8c/results/barcode_fates/plate2_M099d182_14580.csv\x94\x8c/results/barcode_fates/plate2_M099d182_43740.csv\x94\x8c0results/barcode_fates/plate2_M099d182_131220.csv\x94\x8c0results/barcode_fates/plate2_M099d182_393660.csv\x94\x8c+results/barcode_fates/plate2_M099d30_20.csv\x94\x8c+results/barcode_fates/plate2_M099d30_60.csv\x94\x8c,results/barcode_fates/plate2_M099d30_180.csv\x94\x8c,results/barcode_fates/plate2_M099d30_540.csv\x94\x8c-results/barcode_fates/plate2_M099d30_1620.csv\x94\x8c-results/barcode_fates/plate2_M099d30_4860.csv\x94\x8c.results/barcode_fates/plate2_M099d30_14580.csv\x94\x8c.results/barcode_fates/plate2_M099d30_43740.csv\x94\x8c/results/barcode_fates/plate2_M099d30_131220.csv\x94\x8c/results/barcode_fates/plate2_M099d30_393660.csv\x94\x8c*results/barcode_fates/plate2_Y008d0_20.csv\x94\x8c*results/barcode_fates/plate2_Y008d0_60.csv\x94\x8c+results/barcode_fates/plate2_Y008d0_180.csv\x94\x8c+results/barcode_fates/plate2_Y008d0_540.csv\x94\x8c,results/barcode_fates/plate2_Y008d0_1620.csv\x94\x8c,results/barcode_fates/plate2_Y008d0_4860.csv\x94\x8c-results/barcode_fates/plate2_Y008d0_14580.csv\x94\x8c-results/barcode_fates/plate2_Y008d0_43740.csv\x94\x8c.results/barcode_fates/plate2_Y008d0_131220.csv\x94\x8c.results/barcode_fates/plate2_Y008d0_393660.csv\x94\x8c+results/barcode_fates/plate2_Y008d30_20.csv\x94\x8c+results/barcode_fates/plate2_Y008d30_60.csv\x94\x8c,results/barcode_fates/plate2_Y008d30_180.csv\x94\x8c,results/barcode_fates/plate2_Y008d30_540.csv\x94\x8c-results/barcode_fates/plate2_Y008d30_1620.csv\x94\x8c-results/barcode_fates/plate2_Y008d30_4860.csv\x94\x8c.results/barcode_fates/plate2_Y008d30_14580.csv\x94\x8c.results/barcode_fates/plate2_Y008d30_43740.csv\x94\x8c/results/barcode_fates/plate2_Y008d30_131220.csv\x94\x8c/results/barcode_fates/plate2_Y008d30_393660.csv\x94\x8c*results/barcode_fates/plate2_Y154d0_20.csv\x94\x8c*results/barcode_fates/plate2_Y154d0_60.csv\x94\x8c+results/barcode_fates/plate2_Y154d0_180.csv\x94\x8c+results/barcode_fates/plate2_Y154d0_540.csv\x94\x8c,results/barcode_fates/plate2_Y154d0_1620.csv\x94\x8c,results/barcode_fates/plate2_Y154d0_4860.csv\x94\x8c-results/barcode_fates/plate2_Y154d0_14580.csv\x94\x8c-results/barcode_fates/plate2_Y154d0_43740.csv\x94\x8c.results/barcode_fates/plate2_Y154d0_131220.csv\x94\x8c.results/barcode_fates/plate2_Y154d0_393660.csv\x94\x8c,results/barcode_fates/plate2_Y154d182_20.csv\x94\x8c,results/barcode_fates/plate2_Y154d182_60.csv\x94\x8c-results/barcode_fates/plate2_Y154d182_180.csv\x94\x8c-results/barcode_fates/plate2_Y154d182_540.csv\x94\x8c.results/barcode_fates/plate2_Y154d182_1620.csv\x94\x8c.results/barcode_fates/plate2_Y154d182_4860.csv\x94\x8c/results/barcode_fates/plate2_Y154d182_14580.csv\x94\x8c/results/barcode_fates/plate2_Y154d182_43740.csv\x94\x8c0results/barcode_fates/plate2_Y154d182_131220.csv\x94\x8c0results/barcode_fates/plate2_Y154d182_393660.csv\x94\x8c+results/barcode_fates/plate2_Y154d30_20.csv\x94\x8c+results/barcode_fates/plate2_Y154d30_60.csv\x94\x8c,results/barcode_fates/plate2_Y154d30_180.csv\x94\x8c,results/barcode_fates/plate2_Y154d30_540.csv\x94\x8c-results/barcode_fates/plate2_Y154d30_1620.csv\x94\x8c-results/barcode_fates/plate2_Y154d30_4860.csv\x94\x8c.results/barcode_fates/plate2_Y154d30_14580.csv\x94\x8c.results/barcode_fates/plate2_Y154d30_43740.csv\x94\x8c/results/barcode_fates/plate2_Y154d30_131220.csv\x94\x8c/results/barcode_fates/plate2_Y154d30_393660.csv\x94\x8c\'results/barcode_fates/plate2_none-1.csv\x94\x8c\'results/barcode_fates/plate2_none-2.csv\x94\x8c\'results/barcode_fates/plate2_none-3.csv\x94\x8c\'results/barcode_fates/plate2_none-4.csv\x94\x8c\'results/barcode_fates/plate2_none-5.csv\x94\x8c\'results/barcode_fates/plate2_none-6.csv\x94\x8c\'results/barcode_fates/plate2_none-7.csv\x94\x8c\'results/barcode_fates/plate2_none-8.csv\x94\x8c\'results/barcode_fates/plate2_none-9.csv\x94\x8c(results/barcode_fates/plate2_none-10.csv\x94\x8c(results/barcode_fates/plate2_none-11.csv\x94\x8c(results/barcode_fates/plate2_none-12.csv\x94\x8c(results/barcode_fates/plate2_none-13.csv\x94\x8c(results/barcode_fates/plate2_none-14.csv\x94\x8c(results/barcode_fates/plate2_none-15.csv\x94\x8c(results/barcode_fates/plate2_none-16.csv\x94\x8c,results/barcode_invalid/plate2_M099d0_20.csv\x94\x8c,results/barcode_invalid/plate2_M099d0_60.csv\x94\x8c-results/barcode_invalid/plate2_M099d0_180.csv\x94\x8c-results/barcode_invalid/plate2_M099d0_540.csv\x94\x8c.results/barcode_invalid/plate2_M099d0_1620.csv\x94\x8c.results/barcode_invalid/plate2_M099d0_4860.csv\x94\x8c/results/barcode_invalid/plate2_M099d0_14580.csv\x94\x8c/results/barcode_invalid/plate2_M099d0_43740.csv\x94\x8c0results/barcode_invalid/plate2_M099d0_131220.csv\x94\x8c0results/barcode_invalid/plate2_M099d0_393660.csv\x94\x8c.results/barcode_invalid/plate2_M099d182_20.csv\x94\x8c.results/barcode_invalid/plate2_M099d182_60.csv\x94\x8c/results/barcode_invalid/plate2_M099d182_180.csv\x94\x8c/results/barcode_invalid/plate2_M099d182_540.csv\x94\x8c0results/barcode_invalid/plate2_M099d182_1620.csv\x94\x8c0results/barcode_invalid/plate2_M099d182_4860.csv\x94\x8c1results/barcode_invalid/plate2_M099d182_14580.csv\x94\x8c1results/barcode_invalid/plate2_M099d182_43740.csv\x94\x8c2results/barcode_invalid/plate2_M099d182_131220.csv\x94\x8c2results/barcode_invalid/plate2_M099d182_393660.csv\x94\x8c-results/barcode_invalid/plate2_M099d30_20.csv\x94\x8c-results/barcode_invalid/plate2_M099d30_60.csv\x94\x8c.results/barcode_invalid/plate2_M099d30_180.csv\x94\x8c.results/barcode_invalid/plate2_M099d30_540.csv\x94\x8c/results/barcode_invalid/plate2_M099d30_1620.csv\x94\x8c/results/barcode_invalid/plate2_M099d30_4860.csv\x94\x8c0results/barcode_invalid/plate2_M099d30_14580.csv\x94\x8c0results/barcode_invalid/plate2_M099d30_43740.csv\x94\x8c1results/barcode_invalid/plate2_M099d30_131220.csv\x94\x8c1results/barcode_invalid/plate2_M099d30_393660.csv\x94\x8c,results/barcode_invalid/plate2_Y008d0_20.csv\x94\x8c,results/barcode_invalid/plate2_Y008d0_60.csv\x94\x8c-results/barcode_invalid/plate2_Y008d0_180.csv\x94\x8c-results/barcode_invalid/plate2_Y008d0_540.csv\x94\x8c.results/barcode_invalid/plate2_Y008d0_1620.csv\x94\x8c.results/barcode_invalid/plate2_Y008d0_4860.csv\x94\x8c/results/barcode_invalid/plate2_Y008d0_14580.csv\x94\x8c/results/barcode_invalid/plate2_Y008d0_43740.csv\x94\x8c0results/barcode_invalid/plate2_Y008d0_131220.csv\x94\x8c0results/barcode_invalid/plate2_Y008d0_393660.csv\x94\x8c-results/barcode_invalid/plate2_Y008d30_20.csv\x94\x8c-results/barcode_invalid/plate2_Y008d30_60.csv\x94\x8c.results/barcode_invalid/plate2_Y008d30_180.csv\x94\x8c.results/barcode_invalid/plate2_Y008d30_540.csv\x94\x8c/results/barcode_invalid/plate2_Y008d30_1620.csv\x94\x8c/results/barcode_invalid/plate2_Y008d30_4860.csv\x94\x8c0results/barcode_invalid/plate2_Y008d30_14580.csv\x94\x8c0results/barcode_invalid/plate2_Y008d30_43740.csv\x94\x8c1results/barcode_invalid/plate2_Y008d30_131220.csv\x94\x8c1results/barcode_invalid/plate2_Y008d30_393660.csv\x94\x8c,results/barcode_invalid/plate2_Y154d0_20.csv\x94\x8c,results/barcode_invalid/plate2_Y154d0_60.csv\x94\x8c-results/barcode_invalid/plate2_Y154d0_180.csv\x94\x8c-results/barcode_invalid/plate2_Y154d0_540.csv\x94\x8c.results/barcode_invalid/plate2_Y154d0_1620.csv\x94\x8c.results/barcode_invalid/plate2_Y154d0_4860.csv\x94\x8c/results/barcode_invalid/plate2_Y154d0_14580.csv\x94\x8c/results/barcode_invalid/plate2_Y154d0_43740.csv\x94\x8c0results/barcode_invalid/plate2_Y154d0_131220.csv\x94\x8c0results/barcode_invalid/plate2_Y154d0_393660.csv\x94\x8c.results/barcode_invalid/plate2_Y154d182_20.csv\x94\x8c.results/barcode_invalid/plate2_Y154d182_60.csv\x94\x8c/results/barcode_invalid/plate2_Y154d182_180.csv\x94\x8c/results/barcode_invalid/plate2_Y154d182_540.csv\x94\x8c0results/barcode_invalid/plate2_Y154d182_1620.csv\x94\x8c0results/barcode_invalid/plate2_Y154d182_4860.csv\x94\x8c1results/barcode_invalid/plate2_Y154d182_14580.csv\x94\x8c1results/barcode_invalid/plate2_Y154d182_43740.csv\x94\x8c2results/barcode_invalid/plate2_Y154d182_131220.csv\x94\x8c2results/barcode_invalid/plate2_Y154d182_393660.csv\x94\x8c-results/barcode_invalid/plate2_Y154d30_20.csv\x94\x8c-results/barcode_invalid/plate2_Y154d30_60.csv\x94\x8c.results/barcode_invalid/plate2_Y154d30_180.csv\x94\x8c.results/barcode_invalid/plate2_Y154d30_540.csv\x94\x8c/results/barcode_invalid/plate2_Y154d30_1620.csv\x94\x8c/results/barcode_invalid/plate2_Y154d30_4860.csv\x94\x8c0results/barcode_invalid/plate2_Y154d30_14580.csv\x94\x8c0results/barcode_invalid/plate2_Y154d30_43740.csv\x94\x8c1results/barcode_invalid/plate2_Y154d30_131220.csv\x94\x8c1results/barcode_invalid/plate2_Y154d30_393660.csv\x94\x8c)results/barcode_invalid/plate2_none-1.csv\x94\x8c)results/barcode_invalid/plate2_none-2.csv\x94\x8c)results/barcode_invalid/plate2_none-3.csv\x94\x8c)results/barcode_invalid/plate2_none-4.csv\x94\x8c)results/barcode_invalid/plate2_none-5.csv\x94\x8c)results/barcode_invalid/plate2_none-6.csv\x94\x8c)results/barcode_invalid/plate2_none-7.csv\x94\x8c)results/barcode_invalid/plate2_none-8.csv\x94\x8c)results/barcode_invalid/plate2_none-9.csv\x94\x8c*results/barcode_invalid/plate2_none-10.csv\x94\x8c*results/barcode_invalid/plate2_none-11.csv\x94\x8c*results/barcode_invalid/plate2_none-12.csv\x94\x8c*results/barcode_invalid/plate2_none-13.csv\x94\x8c*results/barcode_invalid/plate2_none-14.csv\x94\x8c*results/barcode_invalid/plate2_none-15.csv\x94\x8c*results/barcode_invalid/plate2_none-16.csv\x94\x8c-data/viral_libraries/pdmH1N1_lib2023_loes.csv\x94\x8c3data/neut_standard_sets/loes2023_neut_standards.csv\x94e}\x94(\x8c\x06_names\x94}\x94(\x8c\ncount_csvs\x94K\x00K`\x86\x94\x8c\tfate_csvs\x94K`K\xc0\x86\x94\x8c\x12invalid_count_csvs\x94K\xc0M \x01\x86\x94\x8c\x11viral_library_csv\x94M \x01N\x86\x94\x8c\x15neut_standard_set_csv\x94M!\x01N\x86\x94u\x8c\x12_allowed_overrides\x94]\x94(\x8c\x05index\x94\x8c\x04sort\x94ej;\x01\x00\x00\x8c\tfunctools\x94\x8c\x07partial\x94\x93\x94h\x06\x8c\x19Namedlist._used_attribute\x94\x93\x94\x85\x94R\x94(jA\x01\x00\x00)}\x94\x8c\x05_name\x94j;\x01\x00\x00sNt\x94bj<\x01\x00\x00j?\x01\x00\x00jA\x01\x00\x00\x85\x94R\x94(jA\x01\x00\x00)}\x94jE\x01\x00\x00j<\x01\x00\x00sNt\x94bj/\x01\x00\x00h\x06\x8c\tNamedlist\x94\x93\x94)\x81\x94(h\nh\x0bh\x0ch\rh\x0eh\x0fh\x10h\x11h\x12h\x13h\x14h\x15h\x16h\x17h\x18h\x19h\x1ah\x1bh\x1ch\x1dh\x1eh\x1fh h!h"h#h$h%h&h\'h(h)h*h+h,h-h.h/h0h1h2h3h4h5h6h7h8h9h:h;h<h=h>h?h@hAhBhChDhEhFhGhHhIhJhKhLhMhNhOhPhQhRhShThUhVhWhXhYhZh[h\\h]h^h_h`hahbhchdhehfhghhhie}\x94(j-\x01\x00\x00}\x94j9\x01\x00\x00]\x94(j;\x01\x00\x00j<\x01\x00\x00ej;\x01\x00\x00j?\x01\x00\x00jA\x01\x00\x00\x85\x94R\x94(jA\x01\x00\x00)}\x94jE\x01\x00\x00j;\x01\x00\x00sNt\x94bj<\x01\x00\x00j?\x01\x00\x00jA\x01\x00\x00\x85\x94R\x94(jA\x01\x00\x00)}\x94jE\x01\x00\x00j<\x01\x00\x00sNt\x94bubj1\x01\x00\x00jL\x01\x00\x00)\x81\x94(hjhkhlhmhnhohphqhrhshthuhvhwhxhyhzh{h|h}h~h\x7fh\x80h\x81h\x82h\x83h\x84h\x85h\x86h\x87h\x88h\x89h\x8ah\x8bh\x8ch\x8dh\x8eh\x8fh\x90h\x91h\x92h\x93h\x94h\x95h\x96h\x97h\x98h\x99h\x9ah\x9bh\x9ch\x9dh\x9eh\x9fh\xa0h\xa1h\xa2h\xa3h\xa4h\xa5h\xa6h\xa7h\xa8h\xa9h\xaah\xabh\xach\xadh\xaeh\xafh\xb0h\xb1h\xb2h\xb3h\xb4h\xb5h\xb6h\xb7h\xb8h\xb9h\xbah\xbbh\xbch\xbdh\xbeh\xbfh\xc0h\xc1h\xc2h\xc3h\xc4h\xc5h\xc6h\xc7h\xc8h\xc9e}\x94(j-\x01\x00\x00}\x94j9\x01\x00\x00]\x94(j;\x01\x00\x00j<\x01\x00\x00ej;\x01\x00\x00j?\x01\x00\x00jA\x01\x00\x00\x85\x94R\x94(jA\x01\x00\x00)}\x94jE\x01\x00\x00j;\x01\x00\x00sNt\x94bj<\x01\x00\x00j?\x01\x00\x00jA\x01\x00\x00\x85\x94R\x94(jA\x01\x00\x00)}\x94jE\x01\x00\x00j<\x01\x00\x00sNt\x94bubj3\x01\x00\x00jL\x01\x00\x00)\x81\x94(h\xcah\xcbh\xcch\xcdh\xceh\xcfh\xd0h\xd1h\xd2h\xd3h\xd4h\xd5h\xd6h\xd7h\xd8h\xd9h\xdah\xdbh\xdch\xddh\xdeh\xdfh\xe0h\xe1h\xe2h\xe3h\xe4h\xe5h\xe6h\xe7h\xe8h\xe9h\xeah\xebh\xech\xedh\xeeh\xefh\xf0h\xf1h\xf2h\xf3h\xf4h\xf5h\xf6h\xf7h\xf8h\xf9h\xfah\xfbh\xfch\xfdh\xfeh\xffj\x00\x01\x00\x00j\x01\x01\x00\x00j\x02\x01\x00\x00j\x03\x01\x00\x00j\x04\x01\x00\x00j\x05\x01\x00\x00j\x06\x01\x00\x00j\x07\x01\x00\x00j\x08\x01\x00\x00j\t\x01\x00\x00j\n\x01\x00\x00j\x0b\x01\x00\x00j\x0c\x01\x00\x00j\r\x01\x00\x00j\x0e\x01\x00\x00j\x0f\x01\x00\x00j\x10\x01\x00\x00j\x11\x01\x00\x00j\x12\x01\x00\x00j\x13\x01\x00\x00j\x14\x01\x00\x00j\x15\x01\x00\x00j\x16\x01\x00\x00j\x17\x01\x00\x00j\x18\x01\x00\x00j\x19\x01\x00\x00j\x1a\x01\x00\x00j\x1b\x01\x00\x00j\x1c\x01\x00\x00j\x1d\x01\x00\x00j\x1e\x01\x00\x00j\x1f\x01\x00\x00j \x01\x00\x00j!\x01\x00\x00j"\x01\x00\x00j#\x01\x00\x00j$\x01\x00\x00j%\x01\x00\x00j&\x01\x00\x00j\'\x01\x00\x00j(\x01\x00\x00j)\x01\x00\x00e}\x94(j-\x01\x00\x00}\x94j9\x01\x00\x00]\x94(j;\x01\x00\x00j<\x01\x00\x00ej;\x01\x00\x00j?\x01\x00\x00jA\x01\x00\x00\x85\x94R\x94(jA\x01\x00\x00)}\x94jE\x01\x00\x00j;\x01\x00\x00sNt\x94bj<\x01\x00\x00j?\x01\x00\x00jA\x01\x00\x00\x85\x94R\x94(jA\x01\x00\x00)}\x94jE\x01\x00\x00j<\x01\x00\x00sNt\x94bubj5\x01\x00\x00j*\x01\x00\x00j7\x01\x00\x00j+\x01\x00\x00ub\x8c\x06output\x94h\x06\x8c\x0bOutputFiles\x94\x93\x94)\x81\x94\x8c*results/plates/plate2/frac_infectivity.csv\x94a}\x94(j-\x01\x00\x00}\x94\x8c\x14frac_infectivity_csv\x94K\x00N\x86\x94sj9\x01\x00\x00]\x94(j;\x01\x00\x00j<\x01\x00\x00ej;\x01\x00\x00j?\x01\x00\x00jA\x01\x00\x00\x85\x94R\x94(jA\x01\x00\x00)}\x94jE\x01\x00\x00j;\x01\x00\x00sNt\x94bj<\x01\x00\x00j?\x01\x00\x00jA\x01\x00\x00\x85\x94R\x94(jA\x01\x00\x00)}\x94jE\x01\x00\x00j<\x01\x00\x00sNt\x94bjx\x01\x00\x00ju\x01\x00\x00ub\x8c\x06params\x94h\x06\x8c\x06Params\x94\x93\x94)\x81\x94(\x8c\x12pandas.core.series\x94\x8c\x06Series\x94\x93\x94)\x81\x94}\x94(\x8c\x04_mgr\x94\x8c\x1epandas.core.internals.managers\x94\x8c\x12SingleBlockManager\x94\x93\x94)\x81\x94(]\x94\x8c\x18pandas.core.indexes.base\x94\x8c\n_new_Index\x94\x93\x94\x8c\x19pandas.core.indexes.range\x94\x8c\nRangeIndex\x94\x93\x94}\x94(\x8c\x04name\x94N\x8c\x05start\x94K\x00\x8c\x04stop\x94K`\x8c\x04step\x94K\x01u\x86\x94R\x94a]\x94\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K`\x85\x94j\xa3\x01\x00\x00\x8c\x05dtype\x94\x93\x94\x8c\x02O8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01|\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?t\x94b\x89]\x94(\x8c\x10plate2_M099d0_20\x94\x8c\x10plate2_M099d0_60\x94\x8c\x11plate2_M099d0_180\x94\x8c\x11plate2_M099d0_540\x94\x8c\x12plate2_M099d0_1620\x94\x8c\x12plate2_M099d0_4860\x94\x8c\x13plate2_M099d0_14580\x94\x8c\x13plate2_M099d0_43740\x94\x8c\x14plate2_M099d0_131220\x94\x8c\x14plate2_M099d0_393660\x94\x8c\x12plate2_M099d182_20\x94\x8c\x12plate2_M099d182_60\x94\x8c\x13plate2_M099d182_180\x94\x8c\x13plate2_M099d182_540\x94\x8c\x14plate2_M099d182_1620\x94\x8c\x14plate2_M099d182_4860\x94\x8c\x15plate2_M099d182_14580\x94\x8c\x15plate2_M099d182_43740\x94\x8c\x16plate2_M099d182_131220\x94\x8c\x16plate2_M099d182_393660\x94\x8c\x11plate2_M099d30_20\x94\x8c\x11plate2_M099d30_60\x94\x8c\x12plate2_M099d30_180\x94\x8c\x12plate2_M099d30_540\x94\x8c\x13plate2_M099d30_1620\x94\x8c\x13plate2_M099d30_4860\x94\x8c\x14plate2_M099d30_14580\x94\x8c\x14plate2_M099d30_43740\x94\x8c\x15plate2_M099d30_131220\x94\x8c\x15plate2_M099d30_393660\x94\x8c\x10plate2_Y008d0_20\x94\x8c\x10plate2_Y008d0_60\x94\x8c\x11plate2_Y008d0_180\x94\x8c\x11plate2_Y008d0_540\x94\x8c\x12plate2_Y008d0_1620\x94\x8c\x12plate2_Y008d0_4860\x94\x8c\x13plate2_Y008d0_14580\x94\x8c\x13plate2_Y008d0_43740\x94\x8c\x14plate2_Y008d0_131220\x94\x8c\x14plate2_Y008d0_393660\x94\x8c\x11plate2_Y008d30_20\x94\x8c\x11plate2_Y008d30_60\x94\x8c\x12plate2_Y008d30_180\x94\x8c\x12plate2_Y008d30_540\x94\x8c\x13plate2_Y008d30_1620\x94\x8c\x13plate2_Y008d30_4860\x94\x8c\x14plate2_Y008d30_14580\x94\x8c\x14plate2_Y008d30_43740\x94\x8c\x15plate2_Y008d30_131220\x94\x8c\x15plate2_Y008d30_393660\x94\x8c\x10plate2_Y154d0_20\x94\x8c\x10plate2_Y154d0_60\x94\x8c\x11plate2_Y154d0_180\x94\x8c\x11plate2_Y154d0_540\x94\x8c\x12plate2_Y154d0_1620\x94\x8c\x12plate2_Y154d0_4860\x94\x8c\x13plate2_Y154d0_14580\x94\x8c\x13plate2_Y154d0_43740\x94\x8c\x14plate2_Y154d0_131220\x94\x8c\x14plate2_Y154d0_393660\x94\x8c\x12plate2_Y154d182_20\x94\x8c\x12plate2_Y154d182_60\x94\x8c\x13plate2_Y154d182_180\x94\x8c\x13plate2_Y154d182_540\x94\x8c\x14plate2_Y154d182_1620\x94\x8c\x14plate2_Y154d182_4860\x94\x8c\x15plate2_Y154d182_14580\x94\x8c\x15plate2_Y154d182_43740\x94\x8c\x16plate2_Y154d182_131220\x94\x8c\x16plate2_Y154d182_393660\x94\x8c\x11plate2_Y154d30_20\x94\x8c\x11plate2_Y154d30_60\x94\x8c\x12plate2_Y154d30_180\x94\x8c\x12plate2_Y154d30_540\x94\x8c\x13plate2_Y154d30_1620\x94\x8c\x13plate2_Y154d30_4860\x94\x8c\x14plate2_Y154d30_14580\x94\x8c\x14plate2_Y154d30_43740\x94\x8c\x15plate2_Y154d30_131220\x94\x8c\x15plate2_Y154d30_393660\x94\x8c\rplate2_none-1\x94\x8c\rplate2_none-2\x94\x8c\rplate2_none-3\x94\x8c\rplate2_none-4\x94\x8c\rplate2_none-5\x94\x8c\rplate2_none-6\x94\x8c\rplate2_none-7\x94\x8c\rplate2_none-8\x94\x8c\rplate2_none-9\x94\x8c\x0eplate2_none-10\x94\x8c\x0eplate2_none-11\x94\x8c\x0eplate2_none-12\x94\x8c\x0eplate2_none-13\x94\x8c\x0eplate2_none-14\x94\x8c\x0eplate2_none-15\x94\x8c\x0eplate2_none-16\x94et\x94ba]\x94j\x94\x01\x00\x00j\x97\x01\x00\x00}\x94(j\x99\x01\x00\x00Nj\x9a\x01\x00\x00K\x00j\x9b\x01\x00\x00K`j\x9c\x01\x00\x00K\x01u\x86\x94R\x94a}\x94\x8c\x060.14.1\x94}\x94(\x8c\x04axes\x94j\x91\x01\x00\x00\x8c\x06blocks\x94]\x94}\x94(\x8c\x06values\x94j\xa9\x01\x00\x00\x8c\x08mgr_locs\x94\x8c\x08builtins\x94\x8c\x05slice\x94\x93\x94K\x00K`K\x01\x87\x94R\x94uaust\x94b\x8c\x04_typ\x94\x8c\x06series\x94\x8c\t_metadata\x94]\x94jE\x01\x00\x00a\x8c\x05attrs\x94}\x94\x8c\x06_flags\x94}\x94\x8c\x17allows_duplicate_labels\x94\x88sjE\x01\x00\x00\x8c\x06sample\x94ub}\x94(\x8c\x04date\x94\x8c\n2023-08-01\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate2_samples.csv\x94\x8c\x07samples\x94\x8c\x11pandas.core.frame\x94\x8c\tDataFrame\x94\x93\x94)\x81\x94}\x94(j\x8c\x01\x00\x00j\x8d\x01\x00\x00\x8c\x0cBlockManager\x94\x93\x94(\x8c\x16pandas._libs.internals\x94\x8c\x0f_unpickle_block\x94\x93\x94j\xa2\x01\x00\x00j\xa5\x01\x00\x00K\x00\x85\x94j\xa7\x01\x00\x00\x87\x94R\x94(K\x01K\x02K`\x86\x94j\xaf\x01\x00\x00\x89]\x94(\x8c\x06M099d0\x94jJ\x02\x00\x00jJ\x02\x00\x00jJ\x02\x00\x00jJ\x02\x00\x00jJ\x02\x00\x00jJ\x02\x00\x00jJ\x02\x00\x00jJ\x02\x00\x00jJ\x02\x00\x00\x8c\x08M099d182\x94jK\x02\x00\x00jK\x02\x00\x00jK\x02\x00\x00jK\x02\x00\x00jK\x02\x00\x00jK\x02\x00\x00jK\x02\x00\x00jK\x02\x00\x00jK\x02\x00\x00\x8c\x07M099d30\x94jL\x02\x00\x00jL\x02\x00\x00jL\x02\x00\x00jL\x02\x00\x00jL\x02\x00\x00jL\x02\x00\x00jL\x02\x00\x00jL\x02\x00\x00jL\x02\x00\x00\x8c\x06Y008d0\x94jM\x02\x00\x00jM\x02\x00\x00jM\x02\x00\x00jM\x02\x00\x00jM\x02\x00\x00jM\x02\x00\x00jM\x02\x00\x00jM\x02\x00\x00jM\x02\x00\x00\x8c\x07Y008d30\x94jN\x02\x00\x00jN\x02\x00\x00jN\x02\x00\x00jN\x02\x00\x00jN\x02\x00\x00jN\x02\x00\x00jN\x02\x00\x00jN\x02\x00\x00jN\x02\x00\x00\x8c\x06Y154d0\x94jO\x02\x00\x00jO\x02\x00\x00jO\x02\x00\x00jO\x02\x00\x00jO\x02\x00\x00jO\x02\x00\x00jO\x02\x00\x00jO\x02\x00\x00jO\x02\x00\x00\x8c\x08Y154d182\x94jP\x02\x00\x00jP\x02\x00\x00jP\x02\x00\x00jP\x02\x00\x00jP\x02\x00\x00jP\x02\x00\x00jP\x02\x00\x00jP\x02\x00\x00jP\x02\x00\x00\x8c\x07Y154d30\x94jQ\x02\x00\x00jQ\x02\x00\x00jQ\x02\x00\x00jQ\x02\x00\x00jQ\x02\x00\x00jQ\x02\x00\x00jQ\x02\x00\x00jQ\x02\x00\x00jQ\x02\x00\x00\x8c\x04none\x94jR\x02\x00\x00jR\x02\x00\x00jR\x02\x00\x00jR\x02\x00\x00jR\x02\x00\x00jR\x02\x00\x00jR\x02\x00\x00jR\x02\x00\x00jR\x02\x00\x00jR\x02\x00\x00jR\x02\x00\x00jR\x02\x00\x00jR\x02\x00\x00jR\x02\x00\x00jR\x02\x00\x00\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d0_conc1_S107_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d0_conc2_S115_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d0_conc3_S123_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d0_conc4_S131_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d0_conc5_S139_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d0_conc6_S147_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d0_conc7_S155_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d0_conc8_S163_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d0_conc9_S171_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d0_conc10_S179_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d182_conc1_S105_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d182_conc2_S113_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d182_conc3_S121_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d182_conc4_S129_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d182_conc5_S137_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d182_conc6_S145_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d182_conc7_S153_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d182_conc8_S161_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d182_conc9_S169_R1_001.fastq.gz\x94\x8c\x81/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d182_conc10_S177_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d30_conc1_S106_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d30_conc2_S114_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d30_conc3_S122_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d30_conc4_S130_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d30_conc5_S138_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d30_conc6_S146_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d30_conc7_S154_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d30_conc8_S162_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d30_conc9_S170_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/M099_d30_conc10_S178_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d0_conc1_S112_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d0_conc2_S120_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d0_conc3_S128_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d0_conc4_S136_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d0_conc5_S144_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d0_conc6_S152_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d0_conc7_S160_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d0_conc8_S168_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d0_conc9_S176_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d0_conc10_S184_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d30_conc1_S111_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d30_conc2_S119_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d30_conc3_S127_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d30_conc4_S135_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d30_conc5_S143_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d30_conc6_S151_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d30_conc7_S159_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d30_conc8_S167_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d30_conc9_S175_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y008_d30_conc10_S183_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d0_conc1_S110_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d0_conc2_S118_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d0_conc3_S126_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d0_conc4_S134_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d0_conc5_S142_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d0_conc6_S150_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d0_conc7_S158_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d0_conc8_S166_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d0_conc9_S174_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d0_conc10_S182_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d182_conc1_S108_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d182_conc2_S116_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d182_conc3_S124_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d182_conc4_S132_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d182_conc5_S140_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d182_conc6_S148_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d182_conc7_S156_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d182_conc8_S164_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d182_conc9_S172_R1_001.fastq.gz\x94\x8c\x81/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d182_conc10_S180_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d30_conc1_S109_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d30_conc2_S117_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d30_conc3_S125_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d30_conc4_S133_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d30_conc5_S141_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d30_conc6_S149_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d30_conc7_S157_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d30_conc8_S165_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d30_conc9_S173_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Y154_d30_conc10_S181_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Plate2_Noserum1_S97_R1_001.fastq.gz\x94\x8c\x81/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Plate2_Noserum10_S186_R1_001.fastq.gz\x94\x8c\x81/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Plate2_Noserum11_S187_R1_001.fastq.gz\x94\x8c\x81/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Plate2_Noserum12_S188_R1_001.fastq.gz\x94\x8c\x81/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Plate2_Noserum13_S189_R1_001.fastq.gz\x94\x8c\x81/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Plate2_Noserum14_S190_R1_001.fastq.gz\x94\x8c\x81/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Plate2_Noserum15_S191_R1_001.fastq.gz\x94\x8c\x81/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Plate2_Noserum16_S192_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Plate2_Noserum2_S98_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Plate2_Noserum3_S99_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Plate2_Noserum4_S100_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Plate2_Noserum5_S101_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Plate2_Noserum6_S102_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Plate2_Noserum7_S103_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Plate2_Noserum8_S104_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230801_VH00319_391_AACWKHTM5/Unaligned/Project_aloes/Plate2_Noserum9_S185_R1_001.fastq.gz\x94et\x94bj#\x02\x00\x00K\x00K\x06K\x03\x87\x94R\x94K\x02\x87\x94R\x94jD\x02\x00\x00\x8c\x1apandas.core.arrays.integer\x94\x8c\x0cIntegerArray\x94\x93\x94)\x81\x94}\x94(\x8c\x05_data\x94j\xa2\x01\x00\x00j\xa5\x01\x00\x00K\x00\x85\x94j\xa7\x01\x00\x00\x87\x94R\x94(K\x01K`\x85\x94j\xac\x01\x00\x00\x8c\x02i8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89B\x00\x03\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00<\x00\x00\x00\x00\x00\x00\x00\xb4\x00\x00\x00\x00\x00\x00\x00\x1c\x02\x00\x00\x00\x00\x00\x00T\x06\x00\x00\x00\x00\x00\x00\xfc\x12\x00\x00\x00\x00\x00\x00\xf48\x00\x00\x00\x00\x00\x00\xdc\xaa\x00\x00\x00\x00\x00\x00\x94\x00\x02\x00\x00\x00\x00\x00\xbc\x01\x06\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00<\x00\x00\x00\x00\x00\x00\x00\xb4\x00\x00\x00\x00\x00\x00\x00\x1c\x02\x00\x00\x00\x00\x00\x00T\x06\x00\x00\x00\x00\x00\x00\xfc\x12\x00\x00\x00\x00\x00\x00\xf48\x00\x00\x00\x00\x00\x00\xdc\xaa\x00\x00\x00\x00\x00\x00\x94\x00\x02\x00\x00\x00\x00\x00\xbc\x01\x06\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00<\x00\x00\x00\x00\x00\x00\x00\xb4\x00\x00\x00\x00\x00\x00\x00\x1c\x02\x00\x00\x00\x00\x00\x00T\x06\x00\x00\x00\x00\x00\x00\xfc\x12\x00\x00\x00\x00\x00\x00\xf48\x00\x00\x00\x00\x00\x00\xdc\xaa\x00\x00\x00\x00\x00\x00\x94\x00\x02\x00\x00\x00\x00\x00\xbc\x01\x06\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00<\x00\x00\x00\x00\x00\x00\x00\xb4\x00\x00\x00\x00\x00\x00\x00\x1c\x02\x00\x00\x00\x00\x00\x00T\x06\x00\x00\x00\x00\x00\x00\xfc\x12\x00\x00\x00\x00\x00\x00\xf48\x00\x00\x00\x00\x00\x00\xdc\xaa\x00\x00\x00\x00\x00\x00\x94\x00\x02\x00\x00\x00\x00\x00\xbc\x01\x06\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00<\x00\x00\x00\x00\x00\x00\x00\xb4\x00\x00\x00\x00\x00\x00\x00\x1c\x02\x00\x00\x00\x00\x00\x00T\x06\x00\x00\x00\x00\x00\x00\xfc\x12\x00\x00\x00\x00\x00\x00\xf48\x00\x00\x00\x00\x00\x00\xdc\xaa\x00\x00\x00\x00\x00\x00\x94\x00\x02\x00\x00\x00\x00\x00\xbc\x01\x06\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00<\x00\x00\x00\x00\x00\x00\x00\xb4\x00\x00\x00\x00\x00\x00\x00\x1c\x02\x00\x00\x00\x00\x00\x00T\x06\x00\x00\x00\x00\x00\x00\xfc\x12\x00\x00\x00\x00\x00\x00\xf48\x00\x00\x00\x00\x00\x00\xdc\xaa\x00\x00\x00\x00\x00\x00\x94\x00\x02\x00\x00\x00\x00\x00\xbc\x01\x06\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00<\x00\x00\x00\x00\x00\x00\x00\xb4\x00\x00\x00\x00\x00\x00\x00\x1c\x02\x00\x00\x00\x00\x00\x00T\x06\x00\x00\x00\x00\x00\x00\xfc\x12\x00\x00\x00\x00\x00\x00\xf48\x00\x00\x00\x00\x00\x00\xdc\xaa\x00\x00\x00\x00\x00\x00\x94\x00\x02\x00\x00\x00\x00\x00\xbc\x01\x06\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00<\x00\x00\x00\x00\x00\x00\x00\xb4\x00\x00\x00\x00\x00\x00\x00\x1c\x02\x00\x00\x00\x00\x00\x00T\x06\x00\x00\x00\x00\x00\x00\xfc\x12\x00\x00\x00\x00\x00\x00\xf48\x00\x00\x00\x00\x00\x00\xdc\xaa\x00\x00\x00\x00\x00\x00\x94\x00\x02\x00\x00\x00\x00\x00\xbc\x01\x06\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x94t\x94b\x8c\x05_mask\x94j\xa2\x01\x00\x00j\xa5\x01\x00\x00K\x00\x85\x94j\xa7\x01\x00\x00\x87\x94R\x94(K\x01K`\x85\x94j\xac\x01\x00\x00\x8c\x02b1\x94\x89\x88\x87\x94R\x94(K\x03j\xb0\x01\x00\x00NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89C`\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x94t\x94b\x8c\x06_cache\x94}\x94\x8c\x05dtype\x94j\xb8\x02\x00\x00\x8c\nInt64Dtype\x94\x93\x94)\x81\x94}\x94j\xd4\x02\x00\x00}\x94(\x8c\x0bnumpy_dtype\x94j\xc4\x02\x00\x00\x8c\x04kind\x94\x8c\x01i\x94usbsubj#\x02\x00\x00K\x01K\x02K\x01\x87\x94R\x94K\x02\x87\x94R\x94jD\x02\x00\x00j\xba\x02\x00\x00)\x81\x94}\x94(j\xbd\x02\x00\x00j\xa2\x01\x00\x00j\xa5\x01\x00\x00K\x00\x85\x94j\xa7\x01\x00\x00\x87\x94R\x94(K\x01K`\x85\x94j\xc4\x02\x00\x00\x89B\x00\x03\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00\x07\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\t\x00\x00\x00\x00\x00\x00\x00\n\x00\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x00\x00\x00\x00\x00\x0c\x00\x00\x00\x00\x00\x00\x00\r\x00\x00\x00\x00\x00\x00\x00\x0e\x00\x00\x00\x00\x00\x00\x00\x0f\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x94t\x94bj\xc9\x02\x00\x00j\xa2\x01\x00\x00j\xa5\x01\x00\x00K\x00\x85\x94j\xa7\x01\x00\x00\x87\x94R\x94(K\x01K`\x85\x94j\xd0\x02\x00\x00\x89C`\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x94t\x94bj\xd4\x02\x00\x00}\x94j\xd6\x02\x00\x00j\xd9\x02\x00\x00subj#\x02\x00\x00K\x02K\x03K\x01\x87\x94R\x94K\x02\x87\x94R\x94jD\x02\x00\x00j\xa2\x01\x00\x00j\xa5\x01\x00\x00K\x00\x85\x94j\xa7\x01\x00\x00\x87\x94R\x94(K\x01K\x01K`\x86\x94j\xaf\x01\x00\x00\x89]\x94(jJ\x02\x00\x00jJ\x02\x00\x00jJ\x02\x00\x00jJ\x02\x00\x00jJ\x02\x00\x00jJ\x02\x00\x00jJ\x02\x00\x00jJ\x02\x00\x00jJ\x02\x00\x00jJ\x02\x00\x00jK\x02\x00\x00jK\x02\x00\x00jK\x02\x00\x00jK\x02\x00\x00jK\x02\x00\x00jK\x02\x00\x00jK\x02\x00\x00jK\x02\x00\x00jK\x02\x00\x00jK\x02\x00\x00jL\x02\x00\x00jL\x02\x00\x00jL\x02\x00\x00jL\x02\x00\x00jL\x02\x00\x00jL\x02\x00\x00jL\x02\x00\x00jL\x02\x00\x00jL\x02\x00\x00jL\x02\x00\x00jM\x02\x00\x00jM\x02\x00\x00jM\x02\x00\x00jM\x02\x00\x00jM\x02\x00\x00jM\x02\x00\x00jM\x02\x00\x00jM\x02\x00\x00jM\x02\x00\x00jM\x02\x00\x00jN\x02\x00\x00jN\x02\x00\x00jN\x02\x00\x00jN\x02\x00\x00jN\x02\x00\x00jN\x02\x00\x00jN\x02\x00\x00jN\x02\x00\x00jN\x02\x00\x00jN\x02\x00\x00jO\x02\x00\x00jO\x02\x00\x00jO\x02\x00\x00jO\x02\x00\x00jO\x02\x00\x00jO\x02\x00\x00jO\x02\x00\x00jO\x02\x00\x00jO\x02\x00\x00jO\x02\x00\x00jP\x02\x00\x00jP\x02\x00\x00jP\x02\x00\x00jP\x02\x00\x00jP\x02\x00\x00jP\x02\x00\x00jP\x02\x00\x00jP\x02\x00\x00jP\x02\x00\x00jP\x02\x00\x00jQ\x02\x00\x00jQ\x02\x00\x00jQ\x02\x00\x00jQ\x02\x00\x00jQ\x02\x00\x00jQ\x02\x00\x00jQ\x02\x00\x00jQ\x02\x00\x00jQ\x02\x00\x00jQ\x02\x00\x00\x8c\x06none-1\x94\x8c\x06none-2\x94\x8c\x06none-3\x94\x8c\x06none-4\x94\x8c\x06none-5\x94\x8c\x06none-6\x94\x8c\x06none-7\x94\x8c\x06none-8\x94\x8c\x06none-9\x94\x8c\x07none-10\x94\x8c\x07none-11\x94\x8c\x07none-12\x94\x8c\x07none-13\x94\x8c\x07none-14\x94\x8c\x07none-15\x94\x8c\x07none-16\x94et\x94bj#\x02\x00\x00K\x04K\x05K\x01\x87\x94R\x94K\x02\x87\x94R\x94jD\x02\x00\x00j\xa2\x01\x00\x00j\xa5\x01\x00\x00K\x00\x85\x94j\xa7\x01\x00\x00\x87\x94R\x94(K\x01K\x01K`\x86\x94j\xaf\x01\x00\x00\x89]\x94(\x8c\tM099d0_20\x94\x8c\tM099d0_60\x94\x8c\nM099d0_180\x94\x8c\nM099d0_540\x94\x8c\x0bM099d0_1620\x94\x8c\x0bM099d0_4860\x94\x8c\x0cM099d0_14580\x94\x8c\x0cM099d0_43740\x94\x8c\rM099d0_131220\x94\x8c\rM099d0_393660\x94\x8c\x0bM099d182_20\x94\x8c\x0bM099d182_60\x94\x8c\x0cM099d182_180\x94\x8c\x0cM099d182_540\x94\x8c\rM099d182_1620\x94\x8c\rM099d182_4860\x94\x8c\x0eM099d182_14580\x94\x8c\x0eM099d182_43740\x94\x8c\x0fM099d182_131220\x94\x8c\x0fM099d182_393660\x94\x8c\nM099d30_20\x94\x8c\nM099d30_60\x94\x8c\x0bM099d30_180\x94\x8c\x0bM099d30_540\x94\x8c\x0cM099d30_1620\x94\x8c\x0cM099d30_4860\x94\x8c\rM099d30_14580\x94\x8c\rM099d30_43740\x94\x8c\x0eM099d30_131220\x94\x8c\x0eM099d30_393660\x94\x8c\tY008d0_20\x94\x8c\tY008d0_60\x94\x8c\nY008d0_180\x94\x8c\nY008d0_540\x94\x8c\x0bY008d0_1620\x94\x8c\x0bY008d0_4860\x94\x8c\x0cY008d0_14580\x94\x8c\x0cY008d0_43740\x94\x8c\rY008d0_131220\x94\x8c\rY008d0_393660\x94\x8c\nY008d30_20\x94\x8c\nY008d30_60\x94\x8c\x0bY008d30_180\x94\x8c\x0bY008d30_540\x94\x8c\x0cY008d30_1620\x94\x8c\x0cY008d30_4860\x94\x8c\rY008d30_14580\x94\x8c\rY008d30_43740\x94\x8c\x0eY008d30_131220\x94\x8c\x0eY008d30_393660\x94\x8c\tY154d0_20\x94\x8c\tY154d0_60\x94\x8c\nY154d0_180\x94\x8c\nY154d0_540\x94\x8c\x0bY154d0_1620\x94\x8c\x0bY154d0_4860\x94\x8c\x0cY154d0_14580\x94\x8c\x0cY154d0_43740\x94\x8c\rY154d0_131220\x94\x8c\rY154d0_393660\x94\x8c\x0bY154d182_20\x94\x8c\x0bY154d182_60\x94\x8c\x0cY154d182_180\x94\x8c\x0cY154d182_540\x94\x8c\rY154d182_1620\x94\x8c\rY154d182_4860\x94\x8c\x0eY154d182_14580\x94\x8c\x0eY154d182_43740\x94\x8c\x0fY154d182_131220\x94\x8c\x0fY154d182_393660\x94\x8c\nY154d30_20\x94\x8c\nY154d30_60\x94\x8c\x0bY154d30_180\x94\x8c\x0bY154d30_540\x94\x8c\x0cY154d30_1620\x94\x8c\x0cY154d30_4860\x94\x8c\rY154d30_14580\x94\x8c\rY154d30_43740\x94\x8c\x0eY154d30_131220\x94\x8c\x0eY154d30_393660\x94j\xfb\x02\x00\x00j\xfc\x02\x00\x00j\xfd\x02\x00\x00j\xfe\x02\x00\x00j\xff\x02\x00\x00j\x00\x03\x00\x00j\x01\x03\x00\x00j\x02\x03\x00\x00j\x03\x03\x00\x00j\x04\x03\x00\x00j\x05\x03\x00\x00j\x06\x03\x00\x00j\x07\x03\x00\x00j\x08\x03\x00\x00j\t\x03\x00\x00j\n\x03\x00\x00et\x94bj#\x02\x00\x00K\x05K\x06K\x01\x87\x94R\x94K\x02\x87\x94R\x94jD\x02\x00\x00j\xa2\x01\x00\x00j\xa5\x01\x00\x00K\x00\x85\x94j\xa7\x01\x00\x00\x87\x94R\x94(K\x01K\x01K`\x86\x94j\xaf\x01\x00\x00\x89]\x94(j\xb3\x01\x00\x00j\xb4\x01\x00\x00j\xb5\x01\x00\x00j\xb6\x01\x00\x00j\xb7\x01\x00\x00j\xb8\x01\x00\x00j\xb9\x01\x00\x00j\xba\x01\x00\x00j\xbb\x01\x00\x00j\xbc\x01\x00\x00j\xbd\x01\x00\x00j\xbe\x01\x00\x00j\xbf\x01\x00\x00j\xc0\x01\x00\x00j\xc1\x01\x00\x00j\xc2\x01\x00\x00j\xc3\x01\x00\x00j\xc4\x01\x00\x00j\xc5\x01\x00\x00j\xc6\x01\x00\x00j\xc7\x01\x00\x00j\xc8\x01\x00\x00j\xc9\x01\x00\x00j\xca\x01\x00\x00j\xcb\x01\x00\x00j\xcc\x01\x00\x00j\xcd\x01\x00\x00j\xce\x01\x00\x00j\xcf\x01\x00\x00j\xd0\x01\x00\x00j\xd1\x01\x00\x00j\xd2\x01\x00\x00j\xd3\x01\x00\x00j\xd4\x01\x00\x00j\xd5\x01\x00\x00j\xd6\x01\x00\x00j\xd7\x01\x00\x00j\xd8\x01\x00\x00j\xd9\x01\x00\x00j\xda\x01\x00\x00j\xdb\x01\x00\x00j\xdc\x01\x00\x00j\xdd\x01\x00\x00j\xde\x01\x00\x00j\xdf\x01\x00\x00j\xe0\x01\x00\x00j\xe1\x01\x00\x00j\xe2\x01\x00\x00j\xe3\x01\x00\x00j\xe4\x01\x00\x00j\xe5\x01\x00\x00j\xe6\x01\x00\x00j\xe7\x01\x00\x00j\xe8\x01\x00\x00j\xe9\x01\x00\x00j\xea\x01\x00\x00j\xeb\x01\x00\x00j\xec\x01\x00\x00j\xed\x01\x00\x00j\xee\x01\x00\x00j\xef\x01\x00\x00j\xf0\x01\x00\x00j\xf1\x01\x00\x00j\xf2\x01\x00\x00j\xf3\x01\x00\x00j\xf4\x01\x00\x00j\xf5\x01\x00\x00j\xf6\x01\x00\x00j\xf7\x01\x00\x00j\xf8\x01\x00\x00j\xf9\x01\x00\x00j\xfa\x01\x00\x00j\xfb\x01\x00\x00j\xfc\x01\x00\x00j\xfd\x01\x00\x00j\xfe\x01\x00\x00j\xff\x01\x00\x00j\x00\x02\x00\x00j\x01\x02\x00\x00j\x02\x02\x00\x00j\x03\x02\x00\x00j\x04\x02\x00\x00j\x05\x02\x00\x00j\x06\x02\x00\x00j\x07\x02\x00\x00j\x08\x02\x00\x00j\t\x02\x00\x00j\n\x02\x00\x00j\x0b\x02\x00\x00j\x0c\x02\x00\x00j\r\x02\x00\x00j\x0e\x02\x00\x00j\x0f\x02\x00\x00j\x10\x02\x00\x00j\x11\x02\x00\x00j\x12\x02\x00\x00et\x94bj#\x02\x00\x00K\x06K\x07K\x01\x87\x94R\x94K\x02\x87\x94R\x94jD\x02\x00\x00j\xa2\x01\x00\x00j\xa5\x01\x00\x00K\x00\x85\x94j\xa7\x01\x00\x00\x87\x94R\x94(K\x01K\x01K`\x86\x94j\xaf\x01\x00\x00\x89]\x94(\x8c\x06plate2\x94jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00jy\x03\x00\x00et\x94bj#\x02\x00\x00K\x07K\x08K\x01\x87\x94R\x94K\x02\x87\x94R\x94t\x94]\x94(j\x94\x01\x00\x00j\x92\x01\x00\x00\x8c\x05Index\x94\x93\x94}\x94(\x8c\x04data\x94j\xa2\x01\x00\x00j\xa5\x01\x00\x00K\x00\x85\x94j\xa7\x01\x00\x00\x87\x94R\x94(K\x01K\x08\x85\x94j\xaf\x01\x00\x00\x89]\x94(\x8c\x05serum\x94\x8c\x0fdilution_factor\x94\x8c\treplicate\x94\x8c\x05fastq\x94\x8c\x0fserum_replicate\x94\x8c\x0esample_noplate\x94j0\x02\x00\x00\x8c\x05plate\x94et\x94bj\x99\x01\x00\x00Nu\x86\x94R\x94j\x9e\x01\x00\x00e\x86\x94R\x94j\'\x02\x00\x00\x8c\tdataframe\x94j)\x02\x00\x00]\x94j+\x02\x00\x00}\x94j-\x02\x00\x00}\x94j/\x02\x00\x00\x88subue}\x94(j-\x01\x00\x00}\x94(j:\x02\x00\x00K\x00N\x86\x94\x8c\x0cplate_params\x94K\x01N\x86\x94uj9\x01\x00\x00]\x94(j;\x01\x00\x00j<\x01\x00\x00ej;\x01\x00\x00j?\x01\x00\x00jA\x01\x00\x00\x85\x94R\x94(jA\x01\x00\x00)}\x94jE\x01\x00\x00j;\x01\x00\x00sNt\x94bj<\x01\x00\x00j?\x01\x00\x00jA\x01\x00\x00\x85\x94R\x94(jA\x01\x00\x00)}\x94jE\x01\x00\x00j<\x01\x00\x00sNt\x94bj:\x02\x00\x00j\x8a\x01\x00\x00j\x9d\x03\x00\x00j1\x02\x00\x00ub\x8c\twildcards\x94h\x06\x8c\tWildcards\x94\x93\x94)\x81\x94\x8c\x06plate2\x94a}\x94(j-\x01\x00\x00}\x94\x8c\x05plate\x94K\x00N\x86\x94sj9\x01\x00\x00]\x94(j;\x01\x00\x00j<\x01\x00\x00ej;\x01\x00\x00j?\x01\x00\x00jA\x01\x00\x00\x85\x94R\x94(jA\x01\x00\x00)}\x94jE\x01\x00\x00j;\x01\x00\x00sNt\x94bj<\x01\x00\x00j?\x01\x00\x00jA\x01\x00\x00\x85\x94R\x94(jA\x01\x00\x00)}\x94jE\x01\x00\x00j<\x01\x00\x00sNt\x94bj\x90\x03\x00\x00j\xac\x03\x00\x00ub\x8c\x07threads\x94K\x01\x8c\tresources\x94h\x06\x8c\tResources\x94\x93\x94)\x81\x94(K\x01K\x01\x8c\x15/loc/scratch/34309988\x94e}\x94(j-\x01\x00\x00}\x94(\x8c\x06_cores\x94K\x00N\x86\x94\x8c\x06_nodes\x94K\x01N\x86\x94\x8c\x06tmpdir\x94K\x02N\x86\x94uj9\x01\x00\x00]\x94(j;\x01\x00\x00j<\x01\x00\x00ej;\x01\x00\x00j?\x01\x00\x00jA\x01\x00\x00\x85\x94R\x94(jA\x01\x00\x00)}\x94jE\x01\x00\x00j;\x01\x00\x00sNt\x94bj<\x01\x00\x00j?\x01\x00\x00jA\x01\x00\x00\x85\x94R\x94(jA\x01\x00\x00)}\x94jE\x01\x00\x00j<\x01\x00\x00sNt\x94bj\xc2\x03\x00\x00K\x01j\xc4\x03\x00\x00K\x01j\xc6\x03\x00\x00j\xbf\x03\x00\x00ub\x8c\x03log\x94h\x06\x8c\x03Log\x94\x93\x94)\x81\x94\x8c*results/plates/plate2/process_plate2.ipynb\x94a}\x94(j-\x01\x00\x00}\x94\x8c\x08notebook\x94K\x00N\x86\x94sj9\x01\x00\x00]\x94(j;\x01\x00\x00j<\x01\x00\x00ej;\x01\x00\x00j?\x01\x00\x00jA\x01\x00\x00\x85\x94R\x94(jA\x01\x00\x00)}\x94jE\x01\x00\x00j;\x01\x00\x00sNt\x94bj<\x01\x00\x00j?\x01\x00\x00jA\x01\x00\x00\x85\x94R\x94(jA\x01\x00\x00)}\x94jE\x01\x00\x00j<\x01\x00\x00sNt\x94bj\xd8\x03\x00\x00j\xd5\x03\x00\x00ub\x8c\x06config\x94}\x94(\x8c\x10seqneut-pipeline\x94\x8c\x10seqneut-pipeline\x94\x8c\x06plates\x94}\x94(\x8c\x06plate1\x94}\x94(\x8c\x04date\x94\x8c\x08datetime\x94\x8c\x04date\x94\x93\x94C\x04\x07\xe7\x08\x01\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate1_samples.csv\x94ujy\x03\x00\x00}\x94(j2\x02\x00\x00j\xee\x03\x00\x00C\x04\x07\xe7\x08\x01\x94\x85\x94R\x94j4\x02\x00\x00j5\x02\x00\x00j6\x02\x00\x00j7\x02\x00\x00j8\x02\x00\x00j9\x02\x00\x00u\x8c\x06plate3\x94}\x94(\x8c\x04date\x94j\xee\x03\x00\x00C\x04\x07\xe7\x08\x02\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate3_samples.csv\x94u\x8c\x06plate4\x94}\x94(\x8c\x04date\x94j\xee\x03\x00\x00C\x04\x07\xe7\x08\x02\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate4_samples.csv\x94u\x8c\x06plate5\x94}\x94(\x8c\x04date\x94j\xee\x03\x00\x00C\x04\x07\xe7\x08\x04\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate5_samples.csv\x94u\x8c\x06plate6\x94}\x94(\x8c\x04date\x94j\xee\x03\x00\x00C\x04\x07\xe7\x08\x04\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate6_samples.csv\x94u\x8c\x06plate7\x94}\x94(\x8c\x04date\x94j\xee\x03\x00\x00C\x04\x07\xe7\x08\x05\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate7_samples.csv\x94u\x8c\x06plate8\x94}\x94(\x8c\x04date\x94j\xee\x03\x00\x00C\x04\x07\xe7\x08\x05\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate8_samples.csv\x94u\x8c\x06plate9\x94}\x94(\x8c\x04date\x94j\xee\x03\x00\x00C\x04\x07\xe7\x08\x05\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate9_samples.csv\x94u\x8c\x07plate10\x94}\x94(\x8c\x04date\x94j\xee\x03\x00\x00C\x04\x07\xe7\x08\x06\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1fdata/plates/plate10_samples.csv\x94u\x8c\x07plate11\x94}\x94(\x8c\x04date\x94j\xee\x03\x00\x00C\x04\x07\xe7\t\x1a\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1fdata/plates/plate11_samples.csv\x94u\x8c\x07plate12\x94}\x94(\x8c\x04date\x94j\xee\x03\x00\x00C\x04\x07\xe7\t\x1a\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1fdata/plates/plate12_samples.csv\x94uu\x8c\x0fviral_libraries\x94}\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c-data/viral_libraries/pdmH1N1_lib2023_loes.csv\x94s\x8c\x12neut_standard_sets\x94}\x94\x8c\x08loes2023\x94\x8c3data/neut_standard_sets/loes2023_neut_standards.csv\x94s\x8c\x1eillumina_barcode_parser_params\x94}\x94(\x8c\x08upstream\x94\x8c\x1fCTCCCTACAATGTCGGATTTGTATTTAATAG\x94\x8c\ndownstream\x94\x8c\x00\x94\x8c\x04minq\x94K\x14\x8c\x11upstream_mismatch\x94K\x04\x8c\x0ebc_orientation\x94\x8c\x02R2\x94u\x8c\x17analyze_barcodes_params\x94}\x94\x8c\x10dropdown_columns\x94]\x94\x8c\x05serum\x94asu\x8c\x04rule\x94\x8c\rprocess_plate\x94\x8c\x0fbench_iteration\x94N\x8c\tscriptdir\x94\x8ct/fh/fast/bloom_j/computational_notebooks/jbloom/2023/flu_seqneut_DRIVE_2021-22_repeat_vax/seqneut-pipeline/notebooks\x94ub.'); from snakemake.logging import logger; logger.printshellcmds = False; import os; os.chdir(r'/fh/fast/bloom_j/computational_notebooks/jbloom/2023/flu_seqneut_DRIVE_2021-22_repeat_vax');
######## snakemake preamble end #########


In [3]:
count_csvs = snakemake.input.count_csvs
fate_csvs = snakemake.input.fate_csvs
viral_library_csv = snakemake.input.viral_library_csv
neut_standard_set_csv = snakemake.input.neut_standard_set_csv
frac_infectivity_csv = snakemake.output.frac_infectivity_csv
samples = snakemake.params.samples
plate_params = snakemake.params.plate_params
plate = snakemake.wildcards.plate

samples_df = plate_params["samples"]

assert len(samples) == len(count_csvs) == len(fate_csvs) == len(samples_df)

print(f"Processing {plate=}")

Processing plate='plate2'


## Statistics on barcode-parsing for each sample
Make interactive chart of the "fates" of the sequencing reads parsed for each sample on the plate.

If most sequencing reads are not "valid barcodes", this could potentially indicate some problem in the sequencing or barcode set you are parsing.

Potential fates are:
 - *valid barcode*: barcode that matches a known virus or neutralization standard, we hope most reads are this.
 - *invalid barcode*: a barcode with proper flanking sequences, but does not match a known virus or neutralization standard. If you  have a lot of reads of this type, it is probably a good idea to look at the invalid barcode CSVs (in the `./results/barcode_invalid/` subdirectory created by the pipeline) to see what these invalid barcodes are.
 - *unparseable barcode*: could not parse a barcode from this read as there was not a sequence of the correct length with the appropriate flanking sequence.
 - *low quality barcode*: low-quality or `N` nucleotides in barcode, could indicate problem with sequencing.
 - *failed chastity filter*: reads that failed the Illumina chastity filter, if these are reported in the FASTQ (they may not be).

Also, if the number of reads per sample is very uneven, that could indicate that you did not do a good job of balancing the different samples in the Illumina sequencing.

In [27]:
fates = (
    pd.concat([pd.read_csv(f).assign(sample=s) for f, s in zip(fate_csvs, samples)])
    .merge(samples_df, validate="many_to_one", on="sample")
    .assign(fate_counts=lambda x: x.groupby("fate")["count"].transform("sum"))
    .query("fate_counts > 0")  # only keep fates with at least one count
    [["fate", "count", "serum", "sample_noplate", "dilution_factor"]]
)

assert len(fates) == len(fates.drop_duplicates())

serum_selection = alt.selection_point(
    fields=["serum"],
    bind=alt.binding_select(
        options=[None] + sorted(fates["serum"].unique().tolist()),
        labels=["all"] + [str(s) for s in sorted(fates["serum"].unique())],
        name="serum",
    )
)   

fates_chart = (
    alt.Chart(fates)
    .add_params(serum_selection)
    .transform_filter(serum_selection)
    .encode(
        alt.X("count"),
        alt.Y(
            "sample_noplate",
            title=None,
            sort=list(
                fates.sort_values(["serum", "dilution_factor"])["sample_noplate"]
            ),
        ),
        alt.Color("fate", sort=sorted(fates["fate"].unique(), reverse=True)),
        alt.Order("fate", sort="descending"),
        tooltip=fates.columns.tolist(),
    )
    .mark_bar(height={"band": 0.85})
    .properties(
        height=alt.Step(10),
        width=200,
    )
    .configure_axis(grid=False)
)

fates_chart

alt.Chart(...)